<a href="https://colab.research.google.com/github/petercloud23/Equipe_DataScience_SENAI_Out25/blob/main/UploadToDrive_ChooseFiles_Desktop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tkinter (Interface Desktop Local)

Esta versão abre uma pequena janela para você escolher o arquivo no seu computador e fazer o upload com um clique.

Salve esse código no mesmo diretório onde está seu credentials.json

Quando rodar: Escolha o arquivo no seletor → clique em Enviar para Google Drive → veja a confirmação.

In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import requests # Import requests library

# Define o escopo de permissão necessário para acessar o Google Drive
# Permite que o aplicativo crie e edite arquivos específicos no Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive.file']

def autenticar_google():
    """
    Autentica o usuário no Google Drive e retorna o serviço para interagir com a API.
    Verifica se já existe um token de autenticação salvo localmente.
    Se não existir ou for inválido/expirado, inicia o fluxo de autenticação OAuth 2.0.
    Salva o token de autenticação para uso futuro.
    """
    creds = None
    # Verifica se o arquivo token.json existe (contém credenciais salvas)
    if os.path.exists('token.json'):
        # Carrega as credenciais do arquivo token.json
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Se as credenciais não existirem ou forem inválidas
    if not creds or not creds.valid:
        # Se as credenciais existirem, mas estiverem expiradas e houver um refresh token
        if creds and creds.expired and creds.refresh_token:
            # Tenta renovar as credenciais usando o refresh token
            creds.refresh(Request())
        else:
            # Se não houver credenciais válidas, inicia o fluxo de autenticação
            # Carrega as informações do cliente OAuth 2.0 do arquivo credentials.json
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            # Executa o servidor local para o fluxo de autenticação (abre o navegador para o usuário autenticar)
            creds = flow.run_local_server(port=0)
        # Salva as credenciais (token de acesso e refresh token) no arquivo token.json para uso futuro
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    # Constrói e retorna o serviço da API do Google Drive
    return build('drive', 'v3', credentials=creds)

def upload_arquivo(service, caminho_arquivo):
    """
    Faz upload de um arquivo para o Google Drive.
    Recebe o serviço autenticado do Google Drive e o caminho do arquivo local.
    Verifica se o arquivo existe, obtém o nome do arquivo e faz o upload usando a API.
    Exibe uma mensagem de sucesso ou erro ao usuário.
    """
    # Verifica se o arquivo no caminho especificado existe
    if not os.path.exists(caminho_arquivo):
        # Exibe uma caixa de mensagem de erro se o arquivo não for encontrado
        messagebox.showerror("Erro", f"Arquivo não encontrado: {caminho_arquivo}")
        return

    # Obtém o nome base do arquivo a partir do caminho completo
    nome_arquivo = os.path.basename(caminho_arquivo)
    # Cria um objeto MediaFileUpload para gerenciar o upload do arquivo
    media = MediaFileUpload(caminho_arquivo, resumable=True)
    # Define os metadados do arquivo a ser enviado (neste caso, apenas o nome)
    file_metadata = {'name': nome_arquivo}

    # Chama a API do Google Drive para criar o arquivo
    # body=file_metadata: define o nome do arquivo no Drive
    # media_body=media: especifica o conteúdo do arquivo a ser enviado
    # fields='id': solicita que a resposta inclua apenas o ID do arquivo criado
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    # Exibe uma caixa de mensagem de sucesso com o nome e ID do arquivo enviado
    messagebox.showinfo("Sucesso", f"Arquivo enviado!\nNome: {nome_arquivo}\nID: {file.get('id')}")

def escolher_arquivo():
    """
    Abre uma caixa de diálogo para o usuário selecionar um arquivo local.
    Se um arquivo for selecionado, atualiza o campo de entrada na interface com o caminho do arquivo.
    """
    # Abre a caixa de diálogo para selecionar um arquivo
    caminho = filedialog.askopenfilename(title="Escolha um arquivo para enviar")
    # Verifica se um arquivo foi selecionado (o caminho não é vazio)
    if caminho:
        # Limpa o conteúdo atual do campo de entrada
        entry_caminho.delete(0, tk.END)
        # Insere o caminho do arquivo selecionado no campo de entrada
        entry_caminho.insert(0, caminho)

def enviar_para_drive():
    """
    Obtém o caminho do arquivo do campo de entrada e inicia o processo de upload.
    Primeiro, verifica se um arquivo foi selecionado.
    Em seguida, autentica no Google Drive e chama a função de upload.
    Lida com possíveis erros durante o processo de autenticação ou upload.
    """
    # Obtém o caminho do arquivo digitado ou selecionado no campo de entrada
    caminho = entry_caminho.get()
    # Verifica se o campo de caminho está vazio
    if not caminho:
        # Exibe uma caixa de mensagem de aviso se nenhum arquivo for selecionado
        messagebox.showwarning("Atenção", "Por favor, selecione um arquivo primeiro.")
        return
    try:
        # Tenta autenticar no Google Drive
        service = autenticar_google()
        # Se a autenticação for bem-sucedida, chama a função para fazer o upload do arquivo
        upload_arquivo(service, caminho)
    except Exception as e:
        # Captura qualquer exceção que ocorra durante a autenticação ou upload
        # Exibe uma caixa de mensagem de erro com a descrição da exceção
        messagebox.showerror("Erro", f"Ocorreu um erro:\n{e}")

def desconectar_conta():
    """
    Desconecta a conta Google revogando o token de acesso e removendo o arquivo token.json.
    Verifica se o arquivo token.json existe antes de tentar desconectar.
    Envia uma requisição para a API do Google para revogar o token.
    Remove o arquivo token.json localmente.
    Exibe mensagens informando o resultado da operação.
    """
    # Verifica se o arquivo token.json existe (indica que há uma conta autenticada)
    if os.path.exists("token.json"):
        try:
            # Carrega as credenciais do arquivo local
            creds = Credentials.from_authorized_user_file("token.json")
            # Envia uma requisição POST para a URL de revogação de token do Google OAuth 2.0
            # O token de acesso é passado como parâmetro
            revoke = requests.post(
                'https://oauth2.googleapis.com/revoke',
                params={'token': creds.token},
                headers={'content-type': 'application/x-www-form-urlencoded'}
            )
            # Remove o arquivo token.json do sistema de arquivos local
            os.remove("token.json")
            # Verifica o código de status da resposta da requisição de revogação
            if revoke.status_code == 200:
                # Exibe mensagem de sucesso se a revogação e a remoção local forem bem-sucedidas
                messagebox.showinfo("Sucesso", "Conta desconectada com sucesso!")
            else:
                # Exibe mensagem de aviso se a revogação da API falhar, mas a remoção local for concluída
                messagebox.showwarning("Atenção", "Falha ao revogar token, mas desconexão local concluída.")
        except Exception as e:
            # Captura qualquer exceção que ocorra durante o processo de desconexão
            # Exibe uma caixa de mensagem de erro com a descrição da exceção
            messagebox.showerror("Erro", f"Erro ao desconectar: {e}")
    else:
        # Se o arquivo token.json não existir, informa que nenhuma conta está autenticada
        messagebox.showinfo("Informação", "Nenhuma conta autenticada no momento.")

# --- Configuração da Interface Tkinter ---
# Cria a janela principal da aplicação
janela = tk.Tk()
# Define o título da janela
janela.title("Upload para Google Drive")
# Define o tamanho inicial da janela
janela.geometry("600x300")

# Cria um rótulo (Label) com instruções para o usuário e o adiciona à janela
tk.Label(janela, text="Selecione um arquivo para enviar:", font=("Arial", 12)).pack(pady=10)

# Cria um frame (Frame) para agrupar o campo de entrada e o botão "Procurar"
frame = tk.Frame(janela)
# Adiciona o frame à janela principal
frame.pack(pady=5)

# Cria um campo de entrada (Entry) para exibir/digitar o caminho do arquivo
entry_caminho = tk.Entry(frame, width=50)
# Adiciona o campo de entrada ao frame
entry_caminho.pack(side=tk.LEFT, padx=5)

# Cria um botão "Procurar" (Button) que chama a função escolher_arquivo quando clicado
btn_browse = tk.Button(frame, text="Procurar", command=escolher_arquivo)
# Adiciona o botão "Procurar" ao frame
btn_browse.pack(side=tk.LEFT)

# Cria um botão "Enviar para Google Drive" (Button) que chama a função enviar_para_drive quando clicado
# Define a cor de fundo, cor do texto e fonte do botão
btn_upload = tk.Button(janela, text="Enviar para Google Drive", command=enviar_para_drive, bg="green", fg="white", font=("Arial", 12))
# Adiciona o botão "Enviar para Google Drive" à janela principal
btn_upload.pack(pady=20)

# Cria um botão "Desconectar conta Google" (Button) que chama a função desconectar_conta quando clicado
# Define a cor de fundo, cor do texto e fonte do botão
btn_desconecta = tk.Button(janela, text="Desconectar conta Google", command=desconectar_conta, bg="red", fg="white", font=("Arial", 12))
# Adiciona o botão "Desconectar conta Google" à janela principal
btn_desconecta.pack(pady=20)

# Inicia o loop principal da interface gráfica (mantém a janela aberta e responsiva a eventos)
janela.mainloop()

TclError: no display name and no $DISPLAY environment variable